In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
import os 
import sys 


pd.set_option("display.max_columns", None)

In [9]:
file_path = os.path.join("./", "input.txt")

with open("input.txt", 'r') as f:
    text = f.read()

In [31]:
chars = list(set(text))
vocab_size = len(chars)
print("All characters:\n", "".join(chars))
print("\nVocab size:", vocab_size)

All characters:
 eAK PZ,roim:cMW-IOFsl$daNU!RVHxGfC&wB3YXL;kjtJ'qzEbn.pSDyQTh?
uvg

Vocab size: 65


In [41]:
stoi = {c:i for i, c in enumerate(chars)}
itoi = {i:c for i, c in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda tokens: "".join([itoi[t] for t in tokens])


print(encode("hii there"))
print(decode(encode("hii there")))

[59, 9, 9, 3, 44, 59, 0, 7, 0]
hii there


In [42]:
data = np.array(encode(text))

In [ ]:
len(data)

True